#### 环境配置

In [0]:
!pip install captcha

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
os.chdir('drive/My Drive/4位数字验证码识别')

#### 数字验证码生成

In [0]:
# 生成验证码
from captcha.image import ImageCaptcha
import numpy as np
from PIL import Image
import random, sys

# 错误代码，防止将生成代码再重新执行一遍
print(ss)

number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

def random_captcha_text(char_set=number, captcha_size=4):
    # 验证码列表
    captcha_text = []
    for i in range(captcha_size):
        c = random.choice(char_set)
        captcha_text.append(c)
    return captcha_text

def get_captcha_text_and_image():
    image = ImageCaptcha()
    captcha_text = random_captcha_text()    # 返回的是一个列表，如['3', '4', '5', '6']
    captcha_text = ''.join(captcha_text)    # 将列表拼接成一个字符串
    # 生成验证码
    captcha = image.generate(captcha_text)
    image.write(captcha_text, './images/'+captcha_text+'.jpg')


if __name__=='__main__':
    num = 6000
    for i in range(num):
        get_captcha_text_and_image()
        sys.stdout.write('\r>>Creating image %d, %d'%(i+1, num))    # sys.stdout.write(str) 输出的另一种方式，这种输出不会自动换行
        sys.stdout.flush()    # 保证一秒输出一个结果，要不然上面的结果等程序执行完成后才会输出
    sys.stdout.write('\n')
    sys.stdout.flush()
    print('生成完毕')

>>Creating image 6000, 6000
生成完毕


#### 首次训练模型

#####  模型训练

In [0]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.contrib import slim

# 错误代码，防止将生成代码再重新执行一遍
print(ss)

# 定义基本参数
dataset_dir = './images/'            # 验证码存放文件夹
num_test = 0.2                       # 测试集占20%
batch_size = 64
epochs = 100                         # 训练100个周期
num_classes = 10                     # 分类数为10
lr = tf.Variable(0.001, dtype=tf.float32)    # 学习率
is_training = tf.placeholder(tf.bool)        # 是否为训练状态，tf.bool: True/False


# 获取所有验证码图片路径和标签
def get_images_paths_and_labels(dataset_dir):
    labels = []
    image_path = []
    for filenames in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filenames)
        image_path.append(path)
        image_label = filenames.split('.')[0]    # 如['1234']
        image_labels = []                        # 如[1, 2, 3, 4]
        for i in range(len(image_label)):
            image_labels.append(int(image_label[i]))
        labels.append(image_labels)
    return image_path, labels


image_path, labels = get_images_paths_and_labels(dataset_dir)
# 将得到的列表转换为数组，方便后面的打乱数据
image_path = np.array(image_path)
labels = np.array(labels)
# 打乱数据
np.random.seed(2019)
shuffle_indices = np.random.permutation(np.arange(len(image_path)))    # np.random.permutation()打乱数组，但不在原数组上进行
image_path_shuffled = image_path[shuffle_indices]                      # 按打乱的索引重新取数据，即打乱数据
labels_shuffled = labels[shuffle_indices]
# 切分训练集和测试集
test_sample_index = -1*int(num_test*float(len(image_path)))
x_train, x_test = image_path_shuffled[:test_sample_index], image_path_shuffled[test_sample_index:]
y_train, y_test = labels_shuffled[:test_sample_index], labels_shuffled[test_sample_index:]


# 图像处理函数
def parse_function(filenames, images_labels=None):
    image = tf.read_file(filenames)                    # 读取文件
    image = tf.image.decode_jpeg(image, channels=3)    # 对图像文件解码
    image = tf.image.resize_images(image, [224, 224])  # 将图片重新定义大小，因为后面用的是AlexNet网络，图片大小为224*224
    # 图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image, images_labels


features_placeholder = tf.placeholder(image_path_shuffled.dtype, [None])
labels_placeholder = tf.placeholder(labels_shuffled.dtype, [None, 4])
# 创建dataset对象, 类似于管道机制，需要配合Iterator进行使用
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
dataset = dataset.map(parse_function)    # 处理图片
dataset = dataset.repeat(1)              # 训练周期为1，即将所有数据遍历一次
dataset = dataset.batch(batch_size)      # 批次大小
iterator = dataset.make_initializable_iterator()    # 初始化迭代器
data_batch, label_batch = iterator.get_next()


def AlexNet(inputs, is_training=True):
    '''定义AlexNet神经网络结构，该网络获得2012年的图像识别冠军'''
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu,
                        weights_initializer=tf.glorot_uniform_initializer(),
                        biases_initializer=tf.constant_initializer(0)):    # 会自动初始化所有的权重
        # 首先是常见使用的tf.nn.conv2d的函数，其定义如下：
        # conv2d(input,filter,strides,padding,use_cudnn_on_gpu=None,data_format=None,name=None)
        # 而对于tf.contrib.slim.conv2d，其函数定义如下：
        # convolution(inputs,num_outputs,kernel_size,stride=1,padding='SAME',data_format=None,rate=1,
        #             activation_fn=nn.relu,normalizer_fn=None,normalizer_params=None,
        #             weights_initializer=initializers.xavier_initializer(),weights_regularizer=None,
        #             biases_initializer=init_ops.zeros_initializer(),
        #             biases_regularizer=None,reuse=None,
        #             variables_collections=None,
        #             outputs_collections=None,
        #             trainable=True,scope=None)
        # slim.max_pool2d(inputs,kernel_size,stride=2,padding='VALID',data_format=DATA_FORMAT_NHWC,
        #            outputs_collections=None,scope=None)

#         AlexNet网络，但是精确度不高
#         net = slim.conv2d(inputs, 96, [11, 11], 4)
#         net = slim.max_pool2d(net, [3, 3])
#         net = slim.conv2d(net, 256, [5, 5])
#         net = slim.max_pool2d(net, [3, 3])
#         net = slim.conv2d(net, 384, [5, 5])
#         net = slim.conv2d(net, 384, [3, 3])
#         net = slim.conv2d(net, 256, [3, 3])
#         net = slim.max_pool2d(net, [3, 3])

        # 自定义的神经网络结构
        net = slim.conv2d(inputs, 64, [11, 11], 4)    # 4: stride
        net = slim.max_pool2d(net, [3, 3])
        net = slim.conv2d(net, 192, [5, 5])
        net = slim.max_pool2d(net, [3, 3])
        net = slim.conv2d(net, 384, [3, 3])
        net = slim.conv2d(net, 384, [3, 3])
        net = slim.conv2d(net, 384, [3, 3])
        net = slim.max_pool2d(net, [3, 3])
        
        # 数据扁平化
        net = slim.flatten(net)
        net = slim.fully_connected(net, 4096)
        net = slim.dropout(net, is_training=is_training)

        net0 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net1 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net2 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net3 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)

    return net0, net1, net2, net3


with tf.Session() as sess:
    # 传入数据得到结果
    logit0, logit1, logit2, logit3 = AlexNet(data_batch, is_training)
    # 定义loss
    ## sparse_softmax_cross_etropy: 标签为整数；
    ## softmax_cross_entropy: 标签为one-hot编码
    loss0 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 0], logit0)
    loss1 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 1], logit1)
    loss2 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 2], logit2)
    loss3 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 3], logit3)
    total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0

    # 优化total_loss
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)

    correct0 = tf.nn.in_top_k(logit0, label_batch[:, 0], 1)
    accuracy0 = tf.reduce_mean(tf.cast(correct0, tf.float32))
    correct1 = tf.nn.in_top_k(logit1, label_batch[:, 1], 1)
    accuracy1 = tf.reduce_mean(tf.cast(correct1, tf.float32))
    correct2 = tf.nn.in_top_k(logit2, label_batch[:, 2], 1)
    accuracy2 = tf.reduce_mean(tf.cast(correct2, tf.float32))
    correct3 = tf.nn.in_top_k(logit3, label_batch[:, 3], 1)
    accuracy3 = tf.reduce_mean(tf.cast(correct3, tf.float32))
    total_correct = tf.cast(correct0, tf.float32)*tf.cast(correct1, tf.float32)*tf.cast(correct2, tf.float32)*tf.cast(correct3, tf.float32)
    total_accuracy = tf.reduce_mean(tf.cast(total_correct, tf.float32))

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=5)
    for i in range(epochs):
        if i%30==0:
            sess.run(tf.assign(lr, lr/3))

        sess.run(iterator.initializer, feed_dict={features_placeholder: x_train, labels_placeholder: y_train})
        print('Epoch is '+ str(i))
        print('training...')
        l = 1
        while True:
            try:
                l+=1
                print(l, end=' ')
                sess.run(optimizer, feed_dict={is_training: True})
            except tf.errors.OutOfRangeError:
                # 训练完所有的数据后则跳出循环
                break

        sess.run(iterator.initializer, feed_dict={features_placeholder: x_test, labels_placeholder: y_test})
        print()
        print('testing...')
        j = 1
        while True:
            try:
                j+=1
                print(j, end=' ')
                acc0, acc1, acc2, acc3, total_acc, los = sess.run([accuracy0, accuracy1, accuracy2, accuracy3,
                                                                   total_accuracy, total_loss],
                                                                  feed_dict={is_training: False})
                tf.add_to_collection('sum_losses', los)    # tf.add_to_collection()把多个变量放入一个自己y用引号命名的集合里，也就是把多个变量统一放在一个列表中
                tf.add_to_collection('accuracy0', acc0)
                tf.add_to_collection('accuracy1', acc1)
                tf.add_to_collection('accuracy2', acc2)
                tf.add_to_collection('accuracy3', acc3)
                tf.add_to_collection('total_accuracy', total_acc)
            except tf.errors.OutOfRangeError:
                avg_loss = sess.run(tf.reduce_mean(tf.get_collection('sum_losses')))    # tf.get_collection()从一个结合中取出全部变量，是一个列表
                avg_acc0 = sess.run(tf.reduce_mean(tf.get_collection('accuracy0')))
                avg_acc1 = sess.run(tf.reduce_mean(tf.get_collection('accuracy1')))
                avg_acc2 = sess.run(tf.reduce_mean(tf.get_collection('accuracy2')))
                avg_acc3 = sess.run(tf.reduce_mean(tf.get_collection('accuracy3')))
                avg_total_acc = sess.run(tf.reduce_mean(tf.get_collection('total_accuracy')))
                print('%d:loss=%.3f acc0=%.3f acc1=%.3f acc2=%.3f acc3=%.3f total_acc=%.3f' %(i,avg_loss,avg_acc0,avg_acc1,avg_acc2,avg_acc3,avg_total_acc))
                temp = tf.get_collection_ref('sum_losses')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy0')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy1')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy2')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy3')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('total_accuracy')    # 清空loss统计
                del temp[:]
                break
        saver.save(sess, 'models/model.ckpt')
#         saver.save(sess, 'models/model.ckpt', global_step=10)    # global_step: 在n次迭代后再保存模型

Epoch is 0
training...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
testing...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 0:loss=2.302 acc0=0.103 acc1=0.114 acc2=0.148 acc3=0.094 total_acc=0.000
Epoch is 1
training...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
testing...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 1:loss=2.303 acc0=0.102 acc1=0.095 acc2=0.097 acc3=0.094 total_acc=0.000
Epoch is 2
training...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
testing...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 2:loss=2.303 acc0=0.091 acc1=0.095 acc2=0.097 acc3=0.096 total_acc=0.000
Epoch is 3
training...
2 3 4 5 6 7 8 9 10 11 12 13 14 15 1

###### 总结
训练100个周期后的结果并不是很好，而且耗时较长，可以选择一些更好的网络结构进行模型训练。

###### 总结
使用课程的原代码训练250个周期后，总的准确率达到0.751，虽然效果有所提升，但是在后期，模型提升较慢，这是为什么？？？    
模型迭代次数不够


#### 最后调试代码得到的模型

In [13]:
# 调试自己的代码
import os
import numpy as np
import tensorflow as tf
from tensorflow.contrib import slim

# 错误代码，防止将生成代码再重新执行一遍
print(ss)

# 定义基本参数
dataset_dir = './images/'            # 验证码存放文件夹
num_test = 0.2                       # 测试集占20%
batch_size = 64    # 01不同
epochs = 500                         # 训练100个周期
num_classes = 10                     # 分类数为10
lr = tf.Variable(0.001, dtype=tf.float32)    # 学习率
is_training = tf.placeholder(tf.bool)        # 是否为训练状态，tf.bool: True/False


# 获取所有验证码图片路径和标签
def get_images_paths_and_labels(dataset_dir):
    labels = []
    image_path = []
    for filenames in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filenames)
        image_path.append(path)
        image_label = filenames.split('.')[0]    # 02不同   # 如['1234']
        image_labels = []                        # 如[1, 2, 3, 4]
        for i in range(len(image_label)):
            image_labels.append(int(image_label[i]))
        labels.append(image_labels)
    return image_path, labels


image_path, labels = get_images_paths_and_labels(dataset_dir)
# 将得到的列表转换为数组，方便后面的打乱数据
image_path = np.array(image_path)
labels = np.array(labels)
# 打乱数据
np.random.seed(2019)    # 03不同
shuffle_indices = np.random.permutation(np.arange(len(image_path)))    # np.random.permutation()打乱数组，但不在原数组上进行
image_path_shuffled = image_path[shuffle_indices]                      # 按打乱的索引重新取数据，即打乱数据
labels_shuffled = labels[shuffle_indices]
# 切分训练集和测试集
test_sample_index = -1*int(num_test*float(len(image_path)))
x_train, x_test = image_path_shuffled[:test_sample_index], image_path_shuffled[test_sample_index:]
y_train, y_test = labels_shuffled[:test_sample_index], labels_shuffled[test_sample_index:]


# 图像处理函数
def parse_function(filenames, images_labels=None):
    image = tf.read_file(filenames)                    # 读取文件
    image = tf.image.decode_jpeg(image, channels=3)    # 对图像文件解码
    image = tf.image.resize_images(image, [224, 224])  # 将图片重新定义大小，因为后面用的是AlexNet网络，图片大小为224*224
    # 图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image, images_labels


features_placeholder = tf.placeholder(image_path_shuffled.dtype, [None])
labels_placeholder = tf.placeholder(labels_shuffled.dtype, [None, 4])
# 创建dataset对象, 类似于管道机制，需要配合Iterator进行使用
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
dataset = dataset.map(parse_function)    # 处理图片
dataset = dataset.repeat(1)              # 训练周期为1，即将所有数据遍历一次
dataset = dataset.batch(batch_size)      # 批次大小
iterator = dataset.make_initializable_iterator()    # 初始化迭代器
data_batch, label_batch = iterator.get_next()


def AlexNet(inputs, is_training=True):
    '''定义AlexNet神经网络结构，该网络获得2012年的图像识别冠军'''
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu,
                        weights_initializer=tf.glorot_uniform_initializer(),
                        biases_initializer=tf.constant_initializer(0)):    # 会自动初始化所有的权重

#         AlexNet网络，但是精确度不高
#         net = slim.conv2d(inputs, 96, [11, 11], 4)
#         net = slim.max_pool2d(net, [3, 3])
#         net = slim.conv2d(net, 256, [5, 5])
#         net = slim.max_pool2d(net, [3, 3])
#         net = slim.conv2d(net, 384, [5, 5])
#         net = slim.conv2d(net, 384, [3, 3])
#         net = slim.conv2d(net, 256, [3, 3])
#         net = slim.max_pool2d(net, [3, 3])

        # 自定义的神经网络结构
        net = slim.conv2d(inputs, 64, [11, 11], 4)    # 4: stride
        net = slim.max_pool2d(net, [3, 3])
        net = slim.conv2d(net, 192, [5, 5])
        net = slim.max_pool2d(net, [3, 3])
        net = slim.conv2d(net, 384, [3, 3])
        net = slim.conv2d(net, 384, [3, 3])
        net = slim.conv2d(net, 256, [3, 3])    # 04不同 256
        net = slim.max_pool2d(net, [3, 3])

        # 数据扁平化
        net = slim.flatten(net)
        net = slim.fully_connected(net, 4096)    # 05不同 1024
        net = slim.dropout(net, is_training=is_training)

        net0 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net1 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net2 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)
        net3 = slim.fully_connected(net, num_classes, activation_fn=tf.nn.softmax)

    return net0, net1, net2, net3


with tf.Session() as sess:
    # 传入数据得到结果
    logit0, logit1, logit2, logit3 = AlexNet(data_batch, is_training)
    # 定义loss
    ## sparse_softmax_cross_etropy: 标签为整数；
    ## softmax_cross_entropy: 标签为one-hot编码
    loss0 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 0], logit0)
    loss1 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 1], logit1)
    loss2 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 2], logit2)
    loss3 = tf.losses.sparse_softmax_cross_entropy(label_batch[:, 3], logit3)
    total_loss = (loss0 + loss1 + loss2 + loss3) / 4.0

    # 优化total_loss
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)

    correct0 = tf.nn.in_top_k(logit0, label_batch[:, 0], 1)
    accuracy0 = tf.reduce_mean(tf.cast(correct0, tf.float32))
    correct1 = tf.nn.in_top_k(logit1, label_batch[:, 1], 1)
    accuracy1 = tf.reduce_mean(tf.cast(correct1, tf.float32))
    correct2 = tf.nn.in_top_k(logit2, label_batch[:, 2], 1)
    accuracy2 = tf.reduce_mean(tf.cast(correct2, tf.float32))
    correct3 = tf.nn.in_top_k(logit3, label_batch[:, 3], 1)
    accuracy3 = tf.reduce_mean(tf.cast(correct3, tf.float32))
    total_correct = tf.cast(correct0, tf.float32)*tf.cast(correct1, tf.float32)*tf.cast(correct2, tf.float32)*tf.cast(correct3, tf.float32)
    total_accuracy = tf.reduce_mean(tf.cast(total_correct, tf.float32))

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=5)
    for i in range(epochs):
        if i%150==0:
            sess.run(tf.assign(lr, lr/3))

        sess.run(iterator.initializer, feed_dict={features_placeholder: x_train, labels_placeholder: y_train})
        l = 1
        while True:
            try:
                sess.run(optimizer, feed_dict={is_training: True})
            except tf.errors.OutOfRangeError:
                # 训练完所有的数据后则跳出循环
                break

        sess.run(iterator.initializer, feed_dict={features_placeholder: x_test, labels_placeholder: y_test})

        while True:
            try:
                acc0, acc1, acc2, acc3, total_acc, los = sess.run([accuracy0, accuracy1, accuracy2, accuracy3,
                                                                   total_accuracy, total_loss],
                                                                  feed_dict={is_training: False})
                tf.add_to_collection('sum_losses', los)    # tf.add_to_collection()把多个变量放入一个自己y用引号命名的集合里，也就是把多个变量统一放在一个列表中
                tf.add_to_collection('accuracy0', acc0)
                tf.add_to_collection('accuracy1', acc1)
                tf.add_to_collection('accuracy2', acc2)
                tf.add_to_collection('accuracy3', acc3)
                tf.add_to_collection('total_accuracy', total_acc)
            except tf.errors.OutOfRangeError:
                avg_loss = sess.run(tf.reduce_mean(tf.get_collection('sum_losses')))    # tf.get_collection()从一个结合中取出全部变量，是一个列表
                avg_acc0 = sess.run(tf.reduce_mean(tf.get_collection('accuracy0')))
                avg_acc1 = sess.run(tf.reduce_mean(tf.get_collection('accuracy1')))
                avg_acc2 = sess.run(tf.reduce_mean(tf.get_collection('accuracy2')))
                avg_acc3 = sess.run(tf.reduce_mean(tf.get_collection('accuracy3')))
                avg_total_acc = sess.run(tf.reduce_mean(tf.get_collection('total_accuracy')))
                print('%d:loss=%.3f acc0=%.3f acc1=%.3f acc2=%.3f acc3=%.3f total_acc=%.3f' %(i,avg_loss,avg_acc0,avg_acc1,avg_acc2,avg_acc3,avg_total_acc))
                temp = tf.get_collection_ref('sum_losses')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy0')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy1')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy2')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('accuracy3')    # 清空loss统计
                del temp[:]
                temp = tf.get_collection_ref('total_accuracy')    # 清空loss统计
                del temp[:]
                break
    saver.save(sess, 'models/model.ckpt')    # 07 不同
#         saver.save(sess, 'models/model.ckpt', global_step=10)    # global_step: 在n次迭代后再保存模型

0:loss=2.304 acc0=0.103 acc1=0.093 acc2=0.146 acc3=0.098 total_acc=0.000
1:loss=2.303 acc0=0.097 acc1=0.094 acc2=0.146 acc3=0.091 total_acc=0.000
2:loss=2.303 acc0=0.097 acc1=0.094 acc2=0.146 acc3=0.091 total_acc=0.000
3:loss=2.303 acc0=0.097 acc1=0.094 acc2=0.146 acc3=0.106 total_acc=0.000
4:loss=2.304 acc0=0.097 acc1=0.091 acc2=0.146 acc3=0.091 total_acc=0.000
5:loss=2.305 acc0=0.101 acc1=0.074 acc2=0.104 acc3=0.106 total_acc=0.000
6:loss=2.305 acc0=0.101 acc1=0.094 acc2=0.146 acc3=0.106 total_acc=0.000
7:loss=2.304 acc0=0.087 acc1=0.094 acc2=0.145 acc3=0.114 total_acc=0.000
8:loss=2.258 acc0=0.158 acc1=0.155 acc2=0.221 acc3=0.195 total_acc=0.001
9:loss=2.170 acc0=0.235 acc1=0.254 acc2=0.280 acc3=0.357 total_acc=0.006
10:loss=2.093 acc0=0.375 acc1=0.383 acc2=0.297 acc3=0.387 total_acc=0.015
11:loss=2.003 acc0=0.475 acc1=0.415 acc2=0.431 acc3=0.492 total_acc=0.040
12:loss=1.979 acc0=0.512 acc1=0.419 acc2=0.477 acc3=0.510 total_acc=0.050
13:loss=1.978 acc0=0.527 acc1=0.384 acc2=0.496 a

###### 总结
没有使用AlexNet模型，使用的网络结构相比于AlexNet稍微有些变动，在第250个周期时，这个模型的准确率是0.745，最终完成500个周期时，模型的准确率为0.835，表明训练周期越长，准确率会相对提升一些。

#### 总结
通过训练不同的周期发现，训练周期越长，准确率相对来说就越高；  
在训练的过程中，发现了学习率衰减的问题，具体的可以参考吴恩达深度学习课程中所提到的学习率衰减的办法。